In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
from scipy.ndimage import gaussian_filter
import scipy
from scipy.io import loadmat
import json
from matplotlib import cm as CM
import cv2
import ast
%matplotlib inline

resizing will be done within the ImageDataLoader class on the fly to accomodate varying number of pooling layers

## Function to general labels, given the annotations

In [2]:
def generate_label(label_info, image_shape, dataset):
    """
    Generate a density map based on objects positions.
    Args:
        label_info: list of (x, y, ...) objects positions
        image_shape: (width, height) of a density map to be generated
        dataset: one of {SHT, JHU}
    Returns:
        A density map.
    """    
    # create an empty density map
    density = np.zeros(image_shape, dtype=np.float32)

    height = image_shape[0]
    width = image_shape[1]
    
    if dataset == 'SHT':
        for x, y, *_ in label_info:
            if int(y) < height and int(x) < width:
                density[int(y)][int(x)] = 1
    elif dataset == 'JHU':
        for string in label_info:
            line = string.split()
            x, y = line[0], line[1]
            if int(y) < height and int(x) < width:
                density[int(y)][int(x)] = 1
        
    # apply a convolution with a Gaussian kernel
    density = gaussian_filter(density, sigma=(1, 1), order=0)

    return density

## Generating consolidated HDF5 files for:

#### SHT Part A

In [ ]:
#set the root to the Shanghai dataset you download
root = '../data/SHT/'

part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
path_sets = [part_A_train,part_A_test]

img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

print('Generating HDF5 files...'.format(len(img_paths)))
for id, img_path in enumerate(img_paths):

    gt = io.loadmat(img_path.replace('.jpg','.mat')
                     .replace('images','ground_truth')
                     .replace('IMG_','GT_IMG_'))
    gt = gt['image_info'][0][0][0][0][0]
    
    img = plt.imread(img_path) / 255
    
    resized_image = cv2.resize(img, (768,1024), interpolation = cv2.INTER_CUBIC)
    label = generate_label(gt, img.shape[:2], 'SHT')
    
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','consolidated'), 'w') as hdf:
        hdf['image'] = resized_image
        hdf['density'] = label
        
    print('id {} of {} done'.format(id, len(img_paths)))
        
print('Done: Generated HDF5 files for {} instance(s)'.format(id+1))
print()

#### SHT Part B

In [ ]:
#set the root to the Shanghai dataset you download
root = '../data/SHT/'

part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_B_train,part_B_test]

img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

print('Generating HDF5 files...'.format(len(img_paths)))
for id, img_path in enumerate(img_paths):

    gt = io.loadmat(img_path.replace('.jpg','.mat')
                     .replace('images','ground_truth')
                     .replace('IMG_','GT_IMG_'))
    gt = gt['image_info'][0][0][0][0][0]
    
    img = plt.imread(img_path) / 255
    
    resized_image = cv2.resize(img, (768,1024), interpolation = cv2.INTER_CUBIC)
    label = generate_label(gt, img.shape[:2], 'SHT')
    
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','consolidated'), 'w') as hdf:
        hdf['image'] = img
        hdf['density'] = label
        
    print('id {} of {} done'.format(id, len(img_paths)))
        
print('Done: Generated HDF5 files for {} instance(s)'.format(id+1))
print()

#### JHU

In [5]:
image_labels_path = '../data/JHU/train/image_labels.txt'
val_labels_path = '../data/JHU/val/image_labels.txt'
test_labels_path = '../data/JHU/test/image_labels.txt'
image_labels = {}

first = open(image_labels_path).readlines()
second = open(val_labels_path).readlines()
third = open(test_labels_path).readlines()
alllines = []
alllines.extend(first)
alllines.extend(second)
alllines.extend(third)

for i in range(len(alllines)):
    line = alllines[i].split(',')
    num_ppl = int(line[1])
    weather = int(line[3])
    
    image_labels[line[0]] = {}
    
    # crowd density
    if num_ppl <= 50:
        image_labels[line[0]]['crowd density'] = 'Low'
    elif num_ppl <= 500:
        image_labels[line[0]]['crowd density'] = 'Med'
    else:
        image_labels[line[0]]['crowd density'] = 'High'
        
    # weather
    if weather == 0: 
        image_labels[line[0]]['weather'] = 'None'
    elif weather == 1:
        image_labels[line[0]]['weather'] = 'Fog'
    elif weather == 2:
        image_labels[line[0]]['weather'] = 'Rain'
    else:
        image_labels[line[0]]['weather'] = 'Snow'
        
image_labels

{'0001': {'crowd density': 'Med', 'weather': 'None'},
 '0008': {'crowd density': 'High', 'weather': 'None'},
 '0010': {'crowd density': 'Med', 'weather': 'None'},
 '0012': {'crowd density': 'High', 'weather': 'None'},
 '0014': {'crowd density': 'Med', 'weather': 'None'},
 '0015': {'crowd density': 'Med', 'weather': 'Snow'},
 '0017': {'crowd density': 'Low', 'weather': 'None'},
 '0018': {'crowd density': 'Med', 'weather': 'None'},
 '0019': {'crowd density': 'Low', 'weather': 'None'},
 '0021': {'crowd density': 'Med', 'weather': 'None'},
 '0022': {'crowd density': 'High', 'weather': 'None'},
 '0023': {'crowd density': 'Low', 'weather': 'None'},
 '0025': {'crowd density': 'Low', 'weather': 'Snow'},
 '0026': {'crowd density': 'Low', 'weather': 'Rain'},
 '0027': {'crowd density': 'Med', 'weather': 'None'},
 '0029': {'crowd density': 'Med', 'weather': 'None'},
 '0030': {'crowd density': 'High', 'weather': 'None'},
 '0031': {'crowd density': 'Med', 'weather': 'None'},
 '0032': {'crowd density

In [6]:
#set the root to the JHU dataset you download
root = '../data/JHU/'

if not os.path.exists(os.path.join(root, 'train', 'consolidated')):
    os.mkdir(os.path.join(root, 'train', 'consolidated'))
if not os.path.exists(os.path.join(root, 'test', 'consolidated')):
    os.mkdir(os.path.join(root, 'test', 'consolidated'))
if not os.path.exists(os.path.join(root, 'val', 'consolidated')):
    os.mkdir(os.path.join(root, 'val', 'consolidated'))

JHU_train = os.path.join(root, 'train', 'images')
JHU_test = os.path.join(root, 'test', 'images')
JHU_val = os.path.join(root, 'val', 'images')

path_sets = [JHU_train, JHU_test, JHU_val]

img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

print('Generating HDF5 files...'.format(len(img_paths)))
for id, img_path in enumerate(img_paths):
    
    gt = open(img_path.replace('.jpg', '.txt')
              .replace('images', 'gt'), 'r').readlines()
    
    img = plt.imread(img_path) / 255
    
    label = generate_label(gt, img.shape[:2], 'JHU')
    
    img_id = img_path[-8:-4]
    
    metadata = {
        'img_path': img_id,
        'crowd_density': image_labels[img_id]['crowd density'],
        'weather': image_labels[img_id]['weather'],
    }
    
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','consolidated'), 'w') as hdf:
        hdf['image'] = img
        hdf['density'] = label
        hdf['metadata'] = str(metadata)
        
    print('id {} of {} done'.format(id, len(img_paths)))
    
    #if id+1 == 100: break
        
print('Done: Generated HDF5 files for {} instance(s)'.format(id+1))
print()

Generating HDF5 files...
id 0 of 700 done
id 1 of 700 done
id 2 of 700 done
id 3 of 700 done
id 4 of 700 done
id 5 of 700 done
id 6 of 700 done
id 7 of 700 done
id 8 of 700 done
id 9 of 700 done
id 10 of 700 done
id 11 of 700 done
id 12 of 700 done
id 13 of 700 done
id 14 of 700 done
id 15 of 700 done
id 16 of 700 done
id 17 of 700 done
id 18 of 700 done
id 19 of 700 done
id 20 of 700 done
id 21 of 700 done
id 22 of 700 done
id 23 of 700 done
id 24 of 700 done
id 25 of 700 done
id 26 of 700 done
id 27 of 700 done
id 28 of 700 done
id 29 of 700 done
id 30 of 700 done
id 31 of 700 done
id 32 of 700 done
id 33 of 700 done
id 34 of 700 done
id 35 of 700 done
id 36 of 700 done
id 37 of 700 done
id 38 of 700 done
id 39 of 700 done
id 40 of 700 done
id 41 of 700 done
id 42 of 700 done
id 43 of 700 done
id 44 of 700 done
id 45 of 700 done
id 46 of 700 done
id 47 of 700 done
id 48 of 700 done
id 49 of 700 done
id 50 of 700 done
id 51 of 700 done
id 52 of 700 done
id 53 of 700 done
id 54 of 700 

id 437 of 700 done
id 438 of 700 done
id 439 of 700 done
id 440 of 700 done
id 441 of 700 done
id 442 of 700 done
id 443 of 700 done
id 444 of 700 done
id 445 of 700 done
id 446 of 700 done
id 447 of 700 done
id 448 of 700 done
id 449 of 700 done
id 450 of 700 done
id 451 of 700 done
id 452 of 700 done
id 453 of 700 done
id 454 of 700 done
id 455 of 700 done
id 456 of 700 done
id 457 of 700 done
id 458 of 700 done
id 459 of 700 done
id 460 of 700 done
id 461 of 700 done
id 462 of 700 done
id 463 of 700 done
id 464 of 700 done
id 465 of 700 done
id 466 of 700 done
id 467 of 700 done
id 468 of 700 done
id 469 of 700 done
id 470 of 700 done
id 471 of 700 done
id 472 of 700 done
id 473 of 700 done
id 474 of 700 done
id 475 of 700 done
id 476 of 700 done
id 477 of 700 done
id 478 of 700 done
id 479 of 700 done
id 480 of 700 done
id 481 of 700 done
id 482 of 700 done
id 483 of 700 done
id 484 of 700 done
id 485 of 700 done
id 486 of 700 done
id 487 of 700 done
id 488 of 700 done
id 489 of 70

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))


id 579 of 700 done
id 580 of 700 done
id 581 of 700 done
id 582 of 700 done
id 583 of 700 done
id 584 of 700 done
id 585 of 700 done
id 586 of 700 done
id 587 of 700 done
id 588 of 700 done
id 589 of 700 done
id 590 of 700 done
id 591 of 700 done
id 592 of 700 done
id 593 of 700 done
id 594 of 700 done
id 595 of 700 done
id 596 of 700 done
id 597 of 700 done
id 598 of 700 done
id 599 of 700 done
id 600 of 700 done
id 601 of 700 done
id 602 of 700 done
id 603 of 700 done
id 604 of 700 done
id 605 of 700 done
id 606 of 700 done
id 607 of 700 done
id 608 of 700 done
id 609 of 700 done
id 610 of 700 done
id 611 of 700 done
id 612 of 700 done
id 613 of 700 done
id 614 of 700 done
id 615 of 700 done
id 616 of 700 done
id 617 of 700 done
id 618 of 700 done
id 619 of 700 done
id 620 of 700 done
id 621 of 700 done
id 622 of 700 done
id 623 of 700 done
id 624 of 700 done
id 625 of 700 done
id 626 of 700 done
id 627 of 700 done
id 628 of 700 done
id 629 of 700 done
id 630 of 700 done
id 631 of 70